### Lab: Celery
* create a Celery task named __`write_file()`__ which takes two arguments, a filename and a string, and write the string to the file
* create a second task named __`read_file()`__ which reads from a file
  * if the file does not exist, it waits until the file exists (perhaps have it poll once a second)
* test your tasks


In [1]:
%%file data/celery_examples/file_tasks.py
import os
import time

from celery import Celery

app = Celery('tasks', backend='rpc://', broker='amqp://guest@localhost')

@app.task
def write_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

@app.task
def read_file(filename):
    while not os.path.exists(filename):
        time.sleep(1)
    with open(filename) as f:
        data = f.read()
    return data

Overwriting data/celery_examples/file_tasks.py


In [2]:
import sys
sys.path.append('data')

In [5]:
from celery_examples.file_tasks import write_file, read_file

filename = input('Filename? ')
content = input('Content? ')


result = read_file.delay(filename)
write_file.delay(filename, content)
print(result.get())


Filename? class-celery.txt
Content? Celery is a calorie-free food! (not really)
Celery is a calorie-free food! (not really)


In [6]:
cat data/class-celery.txt

Celery is a calorie-free food! (not really)

In [17]:
rm data/class-celery.txt

In [8]:
result = read_file.delay('class-celery.txt')

In [9]:
result

<AsyncResult: 1f7b09ec-21b8-4c6f-8f58-f214521f2094>

In [10]:
result.ready()

False

In [11]:
write_file.delay('class-celery.txt', 'the stuff')

<AsyncResult: 9f8dd6db-4d3c-4204-9a30-b2c62252dc73>

In [12]:
result.get(timeout=0)

'the stuff'

In [16]:
result.get()

'the stuff'